<a href="https://colab.research.google.com/github/seven320/-Target_on_with_Arduino/blob/master/seven320/extra/save_outer_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このファイルでは外部サイト

https://systematicreviewsjournal.biomedcentral.com/articles?searchType=journalSearch&sort=PubDate&page=4

に載っているシステマティックレビューのtitleとabstractをスクレイピングを用いて抽出します。


In [1]:
from google.colab import drive
from google.colab import output
drive.mount("/content/drive")

!cp /content/drive/MyDrive/signate/medical_paper_judge/input/*.csv ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install Levenshtein > /dev/null

In [3]:
import os, sys

from urllib import request
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
from Levenshtein import distance, ratio
from pathlib import Path

In [4]:
BASE_DIR = "/content/drive/MyDrive/signate/medical_paper_judge"
SUBDIR = Path(os.path.join(BASE_DIR, f"input"))

In [5]:
def get_titles(soup):
    title_elements = soup.find_all(attrs = {"data-test":"title-link"})

    titles = []
    for e in title_elements:
        if e.string is None:
            continue
        titles.append(e.string)
    return titles

In [6]:
def get_abstracts(soup):
    base_url = "https://systematicreviewsjournal.biomedcentral.com"

    abstracts = []
    title_elements = soup.find_all(attrs = {"data-test":"title-link"})
    for e in title_elements:
        if e.string is None:
            continue
        # 論文ごとのページ情報を取得しabstractを抽出
        paper_url = base_url + e.get("href")
        response = request.urlopen(paper_url)
        soup = BeautifulSoup(response)
        response.close()

        a = soup.find_all(attrs = {"property":"og:description"})

        if len(a) == 1:
            abstracts.append(a[0]["content"])
        else:
            abstracts.append("")
        time.sleep(1)

    return abstracts

In [7]:
titles = []
abstracts = []
for i in tqdm(range(45)):
    url = f"https://systematicreviewsjournal.biomedcentral.com/articles?searchType=journalSearch&sort=PubDate&page={i}"

    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()
    time.sleep(1)

    titles += get_titles(soup)
    abstracts += get_abstracts(soup)

    assert len(titles) == len(abstracts)

100%|██████████| 45/45 [1:15:49<00:00, 101.11s/it]


In [8]:
outer_data = pd.DataFrame(np.arange(4 * len(titles)).reshape(-1, 4),
    columns = ["id", "title", "abstract", "judgement"]
)

In [9]:
outer_data["title"] = titles
outer_data["abstract"] = abstracts
outer_data["id"] = np.arange(len(titles))
outer_data["judgement"] = np.ones(len(titles), dtype = np.int8)

In [10]:
outer_data.to_csv(SUBDIR / f"outer_data.csv", index=False)